In [ ]:
#!pip install torch torchtext sentencepiece datasets
# Try opus books dataset for translation - https://huggingface.co/datasets/opus_books

In [ ]:
import numpy as np
import torch
from torch import nn
import functorch
import sys
import os
import math
sys.path.append(os.path.abspath("../../data"))

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 1
SP_VOCAB_SIZE = 1000
CHUNK_LENGTH = 10
Y_CHUNK_LENGTH = 5
TRAIN_SIZE = 500

In [ ]:
from text_data import CNNDatasetWrapper

class Wrapper(CNNDatasetWrapper):
    split_lengths = [TRAIN_SIZE, math.floor(TRAIN_SIZE * .1), 100]
    x_length = CHUNK_LENGTH
    target_length = Y_CHUNK_LENGTH

wrapper = Wrapper(SP_VOCAB_SIZE, DEVICE)

datasets = wrapper.generate_datasets(BATCH_SIZE)
train = datasets["train"]

In [ ]:
class GRUCell(nn.Module):
    def __init__(self, input_units, hidden_units, output_units):
        super(GRUCell, self).__init__()
        self.input_units = input_units
        self.hidden_units = hidden_units
        self.output_units = output_units

        k = math.sqrt(1/hidden_units)
        self.input_weights = nn.Parameter(torch.rand(3, input_units, hidden_units) * 2 * k - k)
        self.input_biases = nn.Parameter(torch.rand(3, 1, hidden_units) * 2 * k - k)

        self.hidden_weights = nn.Parameter(torch.rand(3, hidden_units, hidden_units) * 2 * k - k)
        self.hidden_biases = nn.Parameter(torch.rand(3, 1, hidden_units) * 2 * k - k)

    def forward(self, x, prev_hidden):
        # Compute the regular RNN forward pass
        # Compute update and reset gates for GRU
        reset_gate = torch.sigmoid(x @ self.input_weights[0,] + self.input_biases[0,] + prev_hidden @ self.hidden_weights[0,] + self.hidden_biases[0,])
        update_gate = torch.sigmoid(x @ self.input_weights[1,] + self.input_biases[1,] + prev_hidden @ self.hidden_weights[1,] + self.hidden_biases[1,])
        new_gate = torch.tanh(x @ self.input_weights[2,] + self.input_biases[2,] + torch.mul(reset_gate, prev_hidden @ self.hidden_weights[2,] + self.hidden_biases[2,]))

        hidden_x = torch.mul((1 - update_gate), new_gate) + torch.mul(update_gate, new_gate)
        return hidden_x

In [106]:
class EncoderDecoder(nn.Module):
    def __init__(self, in_sequence_len, out_sequence_len, embedding_len, hidden_units=512, layers=2):
        super(EncoderDecoder, self).__init__()
        self.in_sequence_len = in_sequence_len
        self.out_sequence_len = out_sequence_len
        self.hidden_units = hidden_units
        self.embedding_len = embedding_len
        self.layers = layers

        self.embedding = nn.Embedding(embedding_len, hidden_units)
        self.encoders = nn.ModuleList([GRUCell(input_units=hidden_units, hidden_units=hidden_units, output_units=hidden_units) for _ in range(layers)])
        self.decoders = nn.ModuleList([GRUCell(input_units=hidden_units * 2, hidden_units=hidden_units, output_units=hidden_units) for _ in range(layers)])

        self.linear = nn.Linear(in_features=hidden_units, out_features=embedding_len)

        k = math.sqrt(1/hidden_units)
        self.hidden_attention_weight = nn.Parameter(torch.rand(hidden_units, hidden_units) * 2 * k - k)
        self.context_attention_weight = nn.Parameter(torch.rand(hidden_units, hidden_units) * 2 * k - k)
        self.attention_weight = nn.Parameter(torch.rand(1, hidden_units) * 2 * k - k)
        self.batched_diag = functorch.vmap(torch.diag)

    def attention(self, context, prev_hidden, batch_size):
        # Swap axes so the first dimension of context_attn is batch
        context_attn = torch.bmm(context.swapaxes(0,1), self.context_attention_weight.unsqueeze(0).expand(batch_size,-1,-1))
        # Swap back since prev_hidden is by batch.  This makes the first dim of cross sequence
        cross = torch.tanh(context_attn.swapaxes(0,1) + prev_hidden @ self.hidden_attention_weight)
        # This will be of dimension batch, sequence_length, 1
        attention = torch.bmm(cross.swapaxes(0,1), self.attention_weight.T.unsqueeze(0).expand(batch_size, -1, -1))
        # Drop the last singleton dimension
        attention = attention.squeeze(2)
        # Softmax the predictions across each batch
        probs = torch.softmax(attention, 1)
        diagonalized_probs = self.batched_diag(probs)
        positional_contexts = torch.sum(torch.bmm(diagonalized_probs, context.swapaxes(0,1)), dim=1).reshape(batch_size, self.hidden_units)
        return positional_contexts

    def forward(self, x, y):
        batch_size = x.shape[0]
        # Move batch to the second dimension, so sequence comes first
        y = y.swapaxes(0,1)
        # Embed the input sequence to reduce dimensionality
        embedded = self.embedding(x).swapaxes(0,1)

        # Encode the input sequence
        # Both tensors will have sequence then batch
        enc_hiddens = torch.zeros((1, self.layers, batch_size, self.hidden_units), device=DEVICE)
        for j in range(self.in_sequence_len):
            seq_enc_hiddens = embedded[j,:].unsqueeze(0)
            for i in range(self.layers):
                hidden = self.encoders[i](seq_enc_hiddens[i,], enc_hiddens[j,i])
                # Add first sequence axis
                hidden = hidden.unsqueeze(0)
                seq_enc_hiddens = torch.cat((seq_enc_hiddens, hidden), dim=0)

            enc_hiddens = torch.cat((enc_hiddens, seq_enc_hiddens[1:].unsqueeze(0)), dim=0)

        # Decode to the output sequence
        # Pass in context
        context = enc_hiddens[1:,-1,:,:]
        # Both tensors will have the first dimension be the sequence
        dec_hiddens = torch.zeros(1, self.layers, batch_size, self.hidden_units, device=DEVICE)
        outputs = torch.zeros(1, batch_size, self.embedding_len, device=DEVICE)
        for j in range(self.out_sequence_len):
            # Use either the actual previous y (from the input), or the generated y if the input sequence is shorter than the generation steps.
            if y.shape[0] > j:
                prev_y = y[j,:,:]
            else:
                prev_y = outputs[j,:,:]

            # Run embedding over previous y state
            prev_y = prev_y.argmax(dim=1).int()
            prev_y = self.embedding(prev_y)

            seq_dec_hiddens = prev_y.unsqueeze(0).to(DEVICE)
            for i in range(self.layers):
                positional_context = self.attention(context, dec_hiddens[j,i,], batch_size)
                hidden = self.decoders[i](torch.cat((seq_dec_hiddens[i,], positional_context), dim=1), dec_hiddens[j,i,],)
                # Add first sequence axis
                hidden = hidden.unsqueeze(0)
                seq_dec_hiddens = torch.cat((seq_dec_hiddens, hidden), dim=0)

            # Swap sequence and batch axes to apply linear transform, then swap back
            prev_output = self.linear(seq_dec_hiddens[-1].unsqueeze(0).swapaxes(0,1)).swapaxes(0,1)
            outputs = torch.cat((outputs, prev_output), dim=0)
            dec_hiddens = torch.cat((dec_hiddens, seq_dec_hiddens[1:].unsqueeze(0)), dim=0)

        # Move batch back to axis 0
        out_hiddens = dec_hiddens[1:,-1,:,:].swapaxes(0,1)
        out_output = outputs[1:,].swapaxes(0,1)
        return out_output, out_hiddens

def generate(sequence, target, wrapper):
    pred, _ = model(sequence, target[:,0,:].unsqueeze(1))
    prompts = wrapper.decode_batch(sequence.cpu())
    texts = wrapper.decode_batch(torch.argmax(pred, dim=2).cpu())
    correct_texts = wrapper.decode_batch(torch.argmax(target, dim=2).cpu())

    displays = []
    for p, t, ct in zip(prompts, texts, correct_texts):
        displays.append(f"{p} | {ct} | {t}")
    return displays

In [107]:
from tqdm.auto import tqdm
model = EncoderDecoder(wrapper.x_length, wrapper.target_length, hidden_units=512, layers=1, embedding_len=wrapper.vocab_size).to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
EPOCHS = 1000
DISPLAY_BATCHES = 8

for epoch in range(EPOCHS):
    # Run over the training examples
    train_loss = 0
    match_pct = 0
    for batch, (sequence, target, prev_target) in tqdm(enumerate(train)):
        optimizer.zero_grad()
        forced_target = prev_target
        # Alternate use of teacher forcing vs feeding back own inputs
        if np.random.randint(2) == 0:
            forced_target = prev_target[:,0,:].unsqueeze(1)
        pred, hidden = model(sequence, forced_target)
        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        match_pct += torch.sum(torch.argmax(target, 2) == torch.argmax(pred, 2)) / (Y_CHUNK_LENGTH * BATCH_SIZE)

    with torch.no_grad():
        if epoch % 10 == 0:
            # Show text generated from training prompt as an example
            # Don't feed in all of the train y sequences, just the first token
            # The other y tokens will be predicted by the model and fed back in
            sents = generate(sequence[:DISPLAY_BATCHES], prev_target[:DISPLAY_BATCHES], wrapper)
            for sent in sents:
                print(sent)
            """
            # Compute validation loss.  Unless you have a lot of training data, the validation loss won't decrease.
            valid_loss = 0
            for batch, (sequence, target, prev_target) in enumerate(valid):
                # Only feed in the first token of the actual target
                pred, hidden = model(sequence, prev_target[:,0,:].unsqueeze(1))
                loss = loss_fn(pred, target)
                valid_loss += loss.item()
            print(f"Epoch {epoch} train loss: {train_loss} valid loss: {valid_loss}")
            """
            print(f"Epoch {epoch} train loss: {train_loss} match_pct: {match_pct / len(train)}")

16it [00:02,  7.28it/s]


Girl convicted of killing baby in her | care in S | Lockharte in Saudi
Journalist and two doctors were among t | hose killed . Re | Lockhart killed . Rem
Get a glimpse at | the ne | Lockhart $ex
Head of Iraqi Red Crescent | says parent | Lockhart plotarents
Singapore Airlines | A380 | Lockhart380 super
Bus carrying Marine Corps | Reserve | oneserve un
Russia to send bomber aircraft on lo | ng-range flight | Lockhart-range flights
Barcelona are ready to le | t Brazilian interna | Lockhart Brazilian internation
Epoch 0 train loss: 0.0029501030076062307 match_pct: 0.6515624523162842


16it [00:02,  7.38it/s]
16it [00:02,  7.17it/s]
16it [00:02,  7.40it/s]
16it [00:02,  7.81it/s]
16it [00:02,  7.50it/s]
16it [00:02,  7.47it/s]
16it [00:02,  7.52it/s]
16it [00:02,  7.73it/s]
16it [00:02,  7.60it/s]
16it [00:02,  7.56it/s]


R.E.M. song "U | ntil the | musts sal
Ugandan officer reports tensions | with Liby | W Libyan
Majid Khan's attorneys | say he en | Lockhart he end
Head of Iraqi Red Crescent | says parent | Lockhart parents
U.S. radar picks | up the Russian | mus the Russiane
If you build it, the | tourists will come | W will come to
Glasgow derby betwee | n Celtic | one Celtic and
NEW: "I thought I | was going | Ws going to
Epoch 10 train loss: 0.0025678786041680723 match_pct: 0.6695312857627869


16it [00:02,  7.55it/s]
16it [00:02,  7.56it/s]
16it [00:02,  7.53it/s]
16it [00:02,  7.67it/s]
16it [00:02,  7.58it/s]
16it [00:02,  7.63it/s]
16it [00:02,  7.65it/s]
16it [00:02,  7.55it/s]
16it [00:02,  7.57it/s]
16it [00:02,  7.54it/s]


Singapore Airlines | A380 | Lockhart380 super
Yuvraj Singh becomes | first player to  | W player to hit
Judge: Heather Mills "a  | less than impress | Lockhartargesthan impresss
Paul Burrell, Princess Diana | 's former | Lockharts former but
Experts say there is a dis | parity in | Lockhartarity in h
Soyuz capsule | lands hundred | Wlands hundreds
The car was driving on a | runway at | reache artway sport
"Wine Library TV | " Internet show | W plotet s has
Epoch 20 train loss: 0.002493220497854054 match_pct: 0.670703113079071


5it [00:00,  7.60it/s]